In [ ]:
import numpy as np
import glob, os
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU') #To denoise using CPU for big Maps
from models3 import FRCUnetModel

In [ ]:
import mrcfile

In [ ]:
def standardize(img):
    return ((img-img.mean())/(np.std(img)))

def pad_n(h,factor=32):
    if (h%factor!=0):
        h_new=factor*np.ceil(h/factor)
        return int(h_new)
    return h

def process_output(image,h,w,d):
    image=tf.squeeze(image)
    image=image.numpy()
    image=image[:h,:w,:d]
    return image

def scale_back(out,inp):
    temp=standardize(out)
    temp*=np.std(inp)
    temp+=np.mean(inp)
    return temp

def denoise_map(full_map, model):
    
        dmap=standardize(full_map)
        
        dmap_shape=dmap.shape
        h,w,d=dmap_shape
        h_new=pad_n(h)
        w_new=pad_n(w)
        d_new=pad_n(d)
         
        dmap=np.pad(dmap,((0,h_new-h),(0,w_new-w),(0,d_new-d)),mode='symmetric')

        # Prediction and post-processing
        dmap=model(dmap.reshape([1,h_new,w_new,d_new,1]))
        dmap=process_output(dmap,h,w,d)
        dmap=scale_back(dmap,full_map)
        return dmap
    
def save_map(denoised,half_map1,save_path,voxel_size):

        with mrcfile.new(save_path, overwrite=True) as mrc:
                mrc.set_data(np.float32(denoised))
        with mrcfile.open(save_path,mode='r+') as mrc:
                mrc.set_extended_header(half_map1.header)
                mrc._set_voxel_size(voxel_size.x,voxel_size.y,voxel_size.z)
        print('Map saved')

In [ ]:
model_path = './models/saved-model-epoch-192'
log_path = './logs/'
model = FRCUnetModel(logdir=log_path,model_path=model_path)

In [ ]:
map_num = 11698

In [ ]:
half_map1_path ='./maps/emd_{}_half_map_1.map.gz'.format(map_num)
half_map2_path ='./maps/emd_{}_half_map_2.map.gz'.format(map_num)
mask_path = './maps/emd_{}_msk_1.map'.format(map_num)

In [ ]:
map1_url = "https://ftp.ebi.ac.uk/pub/databases/emdb/structures/EMD-{}/other/emd_{}_half_map_1.map.gz".format(map_num, map_num)
map2_url = "https://ftp.ebi.ac.uk/pub/databases/emdb/structures/EMD-{}/other/emd_{}_half_map_2.map.gz".format(map_num, map_num)
mask_url = "https://ftp.ebi.ac.uk/pub/databases/emdb/structures/EMD-{}/masks/emd_{}_msk_1.map".format(map_num, map_num)

In [ ]:
!wget -P ./maps/ map1_url
!wget -P ./maps/ map2_url
!wget -P ./maps/ mask_url

In [ ]:
!ls ./maps

In [ ]:
half_map1_info = mrcfile.open(half_map1_path)
voxel_size=half_map1_info.voxel_size
half_map1 = mrcfile.open(half_map1_path).data.astype(np.float32)
half_map2 = mrcfile.open(half_map2_path).data.astype(np.float32)
mask_file = mrcfile.open(mask_path).data.astype(np.float32)

In [ ]:
full_map=(half_map1+half_map2)/2

In [ ]:
denoised = denoise_map(full_map, model)

In [ ]:
save_map(denoised,half_map1_info,'./maps/denoised_full_11698.mrc',voxel_size)